Get access token

In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

APP_ID = os.getenv("APP_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
SHORT_TERM_TOKEN = os.getenv("SHORT_TERM_TOKEN")

url = "https://graph.facebook.com/oauth/access_token"

params = {
    "grant_type": "fb_exchange_token",
    "client_id": APP_ID,
    "client_secret": CLIENT_SECRET,
    "fb_exchange_token": SHORT_TERM_TOKEN
}

response = requests.get(url, params=params)

print("Status Code:", response.status_code)
print("Response:", response.json())

# Extract the long-term token
if response.status_code == 200:
    long_term_token = response.json().get("access_token")
    print("Long-term Access Token:", long_term_token)
else:
    print("Error:", response.text)

Status Code: 200
Response: {'access_token': 'EAATD8as9tI0BQZADpo29nFPN3mtPYhztCuiOjCEA3zK9vZATDZCp2HPdq5PANZAZAbHl9Myn5xlic5fHzLFqRPvebnUjZAyZCbncr5Iyrve2iDHV4KHrZCzJhKOLxFzhUc3Ll5gqWANRBHBPS02jwxkpSQZAJGGVN1guQZC5QZAmsGhJPZBOpIRt0dapeO5Xv0Gx', 'token_type': 'bearer', 'expires_in': 5153604}
Long-term Access Token: EAATD8as9tI0BQZADpo29nFPN3mtPYhztCuiOjCEA3zK9vZATDZCp2HPdq5PANZAZAbHl9Myn5xlic5fHzLFqRPvebnUjZAyZCbncr5Iyrve2iDHV4KHrZCzJhKOLxFzhUc3Ll5gqWANRBHBPS02jwxkpSQZAJGGVN1guQZC5QZAmsGhJPZBOpIRt0dapeO5Xv0Gx


Create template

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

# Load environment variables (LONG_TERM_TOKEN and WABA_ID)
load_dotenv()
LONG_TERM_TOKEN = os.getenv("LONG_TERM_TOKEN")
WHATSAPP_BUSINESS_ACCOUNT_ID = os.getenv("WHATSAPP_BUSINESS_ACCOUNT_ID")

# API endpoint for creating a template
url = f"https://graph.facebook.com/v23.0/{WHATSAPP_BUSINESS_ACCOUNT_ID}/message_templates"

# Headers
headers = {
    "Authorization": f"Bearer {LONG_TERM_TOKEN}",
    "Content-Type": "application/json"
}

# Load template JSON from file
with open("msg_template.json", "r", encoding="utf-8") as file:
    payload = json.load(file)

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Print response
print("Status Code:", response.status_code)
print("Response:", response.json())

Status Code: 200
Response: {'id': '2278725515872059', 'status': 'PENDING', 'category': 'UTILITY'}


Check status of template

In [15]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
LONG_TERM_TOKEN = os.getenv("LONG_TERM_TOKEN")
WHATSAPP_BUSINESS_ACCOUNT_ID = os.getenv("WHATSAPP_BUSINESS_ACCOUNT_ID")

# Endpoint to list templates
url = f"https://graph.facebook.com/v23.0/{WHATSAPP_BUSINESS_ACCOUNT_ID}/message_templates"

headers = {
    "Authorization": f"Bearer {LONG_TERM_TOKEN}"
}

# Send GET request
response = requests.get(url, headers=headers)

print("Status Code:", response.status_code)
if response.status_code == 200:
    print("Response:", response.json())
    for template in response.json().get("data", []):
        print("Template Name:", template.get("name"))
        print("   Status:", template.get("status"))
else:
    print("Error:", response.text)

Status Code: 200
Response: {'data': [{'name': 'first_contact', 'parameter_format': 'NAMED', 'components': [{'type': 'BODY', 'text': 'Thank you, {{first_name}}! Your registration is complete.', 'example': {'body_text_named_params': [{'param_name': 'first_name', 'example': 'Pablo'}]}}], 'language': 'it', 'status': 'PENDING', 'category': 'UTILITY', 'id': '2278725515872059'}, {'name': 'jaspers_market_order_confirmation_v1', 'parameter_format': 'POSITIONAL', 'components': [{'type': 'HEADER', 'format': 'TEXT', 'text': 'Order confirmed'}, {'type': 'BODY', 'text': "Hi {{1}},\n\nThank you for your purchase! Your order number is {{2}}.\n\nWe'll start getting your farm fresh groceries ready to ship.\n\nEstimated delivery:  {{3}}.\n\nWe will let you know when your order ships."}, {'type': 'FOOTER', 'text': 'developers.facebook.com'}, {'type': 'BUTTONS', 'buttons': [{'type': 'URL', 'text': 'Visit order details', 'url': 'https://developers.facebook.com/docs/whatsapp/business-management-api/message-t

Send message with template

In [16]:
import requests
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
LONG_TERM_TOKEN = os.getenv("LONG_TERM_TOKEN")
PHONE_NUMBER_ID = os.getenv("PHONE_NUMBER_ID")
RECIPIENT_PHONE = os.getenv("RECIPIENT_PHONE")

# WhatsApp Cloud API endpoint
url = f"https://graph.facebook.com/v22.0/{PHONE_NUMBER_ID}/messages"

# Headers for authentication and content type
headers = {
    "Authorization": f"Bearer {LONG_TERM_TOKEN}",
    "Content-Type": "application/json"
}

# Message payload
payload = {
    "messaging_product": "whatsapp",
    "to": RECIPIENT_PHONE,
    "type": "template",
    "template": {
        "name": "first_contact",
        "language": {
            "code": "it"
        }
    },
    "components": [
        {
            "type": "body",
            "parameters": [
                {
                    "type": "text",
                    "parameter_name": "first_name",
                    "text": "Adriano"
                }
            ]
        }
    ]
}

# Send the POST request with timeout
try:
    response = requests.post(url, headers=headers, json=payload, verify=True, timeout=30)
    print("Status Code:", response.status_code)
    if response.status_code == 200:
        try:
            print("Response:", response.json())
        except ValueError:
            print("Response Text:", response.text)
    else:
        print("Response Text:", response.text)
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Status Code: 404
Response Text: {"error":{"message":"(#132001) Template name does not exist in the translation","type":"OAuthException","code":132001,"error_data":{"messaging_product":"whatsapp","details":"template name (first_contact) does not exist in it"},"fbtrace_id":"Ab5SZCK5QbRIxScHHY9OHRs"}}
